# 第１ステージ　ステップ1-10

In [1]:
import numpy as np

## ステップ１

### 1.1変数とは
- 省略

### 1.2 Variableクラスの実装

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data

In [3]:
data = np.array(1.0)
x = Variable(data)
x.data

array(1.)

In [4]:
x.data = np.array(2.)
x.data

array(2.)

### 1.3 【補足】Nunpyの多次元配列
- 省略

## ステップ２

### 2.2 Functionクラスの実装

In [6]:
class Function:
    def __call__(self, input):
        x = input.data # データを取り出す
        y = x ** 2 # 実際の計算
        output = Variable(y) # Variableとして返す
        return output

In [7]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [8]:
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


### 2.3 Functionクラスを使う

1. Functionクラスは基底クラスとして、全ての関数に共通する機能を実装する
2. 具体的な関数は、Functionクラスを継承したクラスで実装する

以上の点を考慮し、Functionクラスを実装

In [9]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 具体的な計算はforwardメソッドで行う。
        output = Variable(y)
        return output
    
    def forward(self, x):
        # 例外処理を行い、このクラスを継承すべきであるとアピール
        raise NotImplementedError()

- Functionクラスを継承して、Squareクラスを実装してみる

In [10]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [12]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## ステップ３　関数の連結

### 3.1 Exp関数の実装

- Functionクラスを継承して、$y=e^x$を実装

In [13]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

### 関数を連結する
- Functionクラスを継承したクラスを連結して、$y=(e^{x^2})^2$を実装
<br>
- 実装の流れ<br>
$x$ &rArr; Square() &rArr; $x^2$ &rArr; Exp() &rArr; $e^{x^2}$ &rArr; Square() &rArr; $(e^{x^2})^2$

In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x) # x^2
b = B(a) # e^(x^2)
y = C(b) # (e^(x^2))^2
print(y.data)

1.648721270700128
